# Generate new colors characteristics from google sheet specification
20200608 SEARCH-by strat. representation-Bin. obs_v1

In [2]:
import colorsys
import re
from  colorsys import rgb_to_hls
from IPython.core.display import display, HTML
from pprint import pprint
import csv

# spec order
# main_colors=['Yellow','Orange','Pink', Magenta','Red','Green','Blue','Brown','Grey']
#

# color wheel order (additive synthesis)
# main_colors=['Red','Orange','Yellow','Brown','Green','Blue','Magenta','Pink','Grey']
# chromatic circle (soustractive synthesis)
main_colors=['Yellow','Orange','Brown','Red','Pink','Magenta','Blue','Turquoise','Green','Grey']

rgb_colors={'Yellow':(255,255,0),'Orange':(255,165,0),'Magenta':(255,0,255),
            'Red':(255,0,0),'Green':(0,255,0),'Blue':(0,0,255),
            'Brown':(165,42,42),'Grey':(128,128,128),
           'LightBlue':(173,216,230),'MediumBlue':(0,0,205), 'DarkBlue':(0,0,139),
           'lightgreen':(144,238,144),'darkgreen':(0,100,0),
           'Pink':(255,192,203)}#FFC0CB

def hls_to_csshsl(hls):
    return (round(hls[0]*360),round(100*hls[2]),round(100*hls[1]))

for c in rgb_colors:
    hls = rgb_to_hls(*(c255/255 for c255 in rgb_colors[c]))
    print(c,rgb_colors[c],hls_to_csshsl(hls) )

hsl_colors={
            'Red':(0, 100, 50),
            'Brown':(0, 60, 40),
            'Orange':(30, 100, 50),
            'Yellow':(60, 100, 50),
            'Green':(120, 100, 50),
            'Blue':(240, 100, 50),
            'Magenta':(300, 100, 50),
            'Pink':(350, 100, 88),
            'Turquoise': (174, 72, 56),
            'Grey':(0,0,50)}


order =2

# build html table to preview colors
html='''
<h1>Colors preview</h1>
<table style="border-spacing:10px;border-collapse:separate">
    <thead>
    <tr>
        <th>uid</th> <th>color</th> <th>css</th>
    </tr>
</thead>
<tbody>'''

def new_color_characteristic(optgroup, uid,name,color,description, order):
    global html
    color_str = f'hsl({color[0]}, {color[1]}%, {color[2]}%)'
    characteristic = dict(optgroup=optgroup, uid=uid,name=name,color=color_str,description=description,order=order)
    print(characteristic,',')
    html+=f'''
    <tr>
        <td>{uid}</td><td style="background-color:{color_str};height:50px">{name}</td><td>{color_str}</td>
    </tr>'''


csv_file_path='./data/new_colors.csv'
with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)
    old2new = {}
    newcolor = {}
    for r in reader:
        print(' '.join([f"{k}:{r[k]}" for k in r]))
        if r['rgb'] and r['newrgb']:
            old2new[r['rgb']] = r['newrgb']
        if r['name'][0]=='*':
            newcolor[r['name'][1:]]=r['newrgb']

def get_colors_from_rgbstring(rgb_string):
    color_rgb = [int(rgb_string[n:n+2],16) for n in range(1,7,2)]
    color_hls = colorsys.rgb_to_hls(*[c/255 for c in color_rgb])
    color_hsl = hls_to_csshsl(color_hls)
    return color_rgb, color_hsl

for c in main_colors:
    for v in ['Light','','Dark']:
        coefs={'Light':{'s':.75,'l':1.5},
              'Medium':{'s':1,'l':1},
              'Dark':{'s':1,'l':.5}}
        uid = (c.lower() if v=='' else v.lower()+c)+'Characteristic'
        description=v.lower()+c.lower()
        name = v + (' ' if v!='' else '') + c
        color = hsl_colors[c]
        if v != '':
            color=(color[0],color[1]*coefs[v]['s'],color[2]*coefs[v]['l'])
            if coefs[v]['l']>1 and color[2]>95:
                color=(color[0],color[1]*coefs[v]['s'],95)
        color_rgb = (int(255*cpnt) for cpnt in colorsys.hls_to_rgb(color[0]/360,color[2]/100,color[1]/100))
        description = "#{:02x}{:02x}{:02x}".format(*color_rgb)

        if description in old2new:
            old_color=color
            old_description=description
            description=old2new[description].lower() #description = html rgb string
            new_color_rgb, color = get_colors_from_rgbstring(description)
            print(f"//old_rgb={old_description} new rgb:{description} => {new_color_rgb} old_hls:{old_color} => {color}")

        new_color_characteristic(optgroup=c, uid=uid,name=name,color=color,description=description,order=order)
        order += 1

# add new colors
def to_uid(name):
    camel_case_name=''.join(name.title().split(' '))
    return camel_case_name[0].lower() + camel_case_name[1:]

for name,rgb_string in newcolor.items():
    color_rgb, color = get_colors_from_rgbstring(rgb_string)
    new_color_characteristic(optgroup=' ', uid=to_uid(name + ' characteristic'),name=name,color=color,description=rgb_string,order=order)
    order += 1


html+='</tbody></table>'


display(HTML(html))



Yellow (255, 255, 0) (60, 100, 50)
Orange (255, 165, 0) (39, 100, 50)
Magenta (255, 0, 255) (300, 100, 50)
Red (255, 0, 0) (0, 100, 50)
Green (0, 255, 0) (120, 100, 50)
Blue (0, 0, 255) (240, 100, 50)
Brown (165, 42, 42) (0, 59, 41)
Grey (128, 128, 128) (0, 0, 50)
LightBlue (173, 216, 230) (195, 53, 79)
MediumBlue (0, 0, 205) (240, 100, 40)
DarkBlue (0, 0, 139) (240, 100, 27)
lightgreen (144, 238, 144) (120, 73, 75)
darkgreen (0, 100, 0) (120, 100, 20)
Pink (255, 192, 203) (350, 100, 88)
['name', 'rgb', 'newrgb']
name:light yellow rgb:#efef8f newrgb:#FFF8DC
name:yellow rgb:#feff00 newrgb:None
name:dark yellow rgb:#7f7f00 newrgb:#FFD700
name:light orange rgb:#efbf8f newrgb:None
name:orange rgb:#ff7f00 newrgb:None
name:dark orange rgb:#7f3f00 newrgb:#D2691E
name:light brown rgb:#c66b6b newrgb:#DEB887
name:brown rgb:#a32828 newrgb:#8B4513
name:dark brown rgb:#511414 newrgb:None
name:light red rgb:#ef8f8f newrgb:#FFCBC5
name:red rgb:#ff0000 newrgb:None
name:dark red rgb:#7f0000 newrgb:#B22

uid,color,css
lightYellowCharacteristic,Light Yellow,"hsl(48, 100%, 93%)"
yellowCharacteristic,Yellow,"hsl(60, 100%, 50%)"
darkYellowCharacteristic,Dark Yellow,"hsl(51, 100%, 50%)"
lightOrangeCharacteristic,Light Orange,"hsl(30, 75.0%, 75.0%)"
orangeCharacteristic,Orange,"hsl(30, 100%, 50%)"
darkOrangeCharacteristic,Dark Orange,"hsl(25, 75%, 47%)"
lightBrownCharacteristic,Light Brown,"hsl(34, 57%, 70%)"
brownCharacteristic,Brown,"hsl(25, 76%, 31%)"
darkBrownCharacteristic,Dark Brown,"hsl(0, 60%, 20.0%)"
lightRedCharacteristic,Light Red,"hsl(6, 100%, 89%)"


In [4]:
get_colors_from_rgbstring('#ffffff')

([255, 255, 255], (0, 0, 100))

In [58]:
newcolor.items()

dict_items([('pale turquoise', '#AFEEEE'), ('turquoise', '#40E0D0'), ('dark turquoise', '#00CED1'), ('extra light grey', None), ('dark grey', '#575757'), ('extra dark grey', '#3f3f3f')])

In [7]:
['medium'+ c for  c in main_colors]


['mediumRed',
 'mediumOrange',
 'mediumYellow',
 'mediumBrown',
 'mediumGreen',
 'mediumBlue',
 'mediumMagenta',
 'mediumPink',
 'mediumGrey']

In [50]:
description
old2new
for i in range(1,7,2):
    print(i)
"FF".lower()
import json
json.dumps(characteristic)
jsom.dumps(

1
3
5


'{"optgroup": "Grey", "uid": "darkGreyCharacteristic", "name": "Dark Grey", "color": "hsl(0, 0%, 25.0%)", "description": "#3f3f3f", "order": 28}'

In [8]:
html='''
<h1>12 main colors (30 degrees steps)</h1>
<table style="border-spacing:10px;border-collapse:separate">
    <thead>
    <tr>
        <th>hue</th> <th>color</th>
    </tr>
</thead>
<tbody>'''

# 12 primary,secondary and tertiary colors (every 30 degrees on color wheel)
for c in range(0,360,30):
        color_str = f'hsl({c}, 100%, 50%)'
        html+=f'''
        <tr>
            <td>{c}</td> <td style="background-color:{color_str}; height:50px; vertical-align:top">
            <span style="background-color:white; display:block">{color_str}</span>
            </td>
        </tr>'''

html+='</tbody></table>'


display(HTML(html))

hue,color
0,"hsl(0, 100%, 50%)"
30,"hsl(30, 100%, 50%)"
60,"hsl(60, 100%, 50%)"
90,"hsl(90, 100%, 50%)"
120,"hsl(120, 100%, 50%)"
150,"hsl(150, 100%, 50%)"
180,"hsl(180, 100%, 50%)"
210,"hsl(210, 100%, 50%)"
240,"hsl(240, 100%, 50%)"
270,"hsl(270, 100%, 50%)"


In [39]:
int("#FFF8dc"[3:5],16)



248

In [9]:
import csv
csv_file_path='./old_colors.csv'


html='''
<h1>existing MiCorr colors</h1>
<table style="border-spacing:10px;border-collapse:separate">
    <thead>
    <tr>
        <th>uid</th> <th>name</th> <th>color</th>
    </tr>
</thead>
<tbody>'''

# 12 primary,secondary and tertiary colors (every 30 degrees on color wheel)
with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)
    for r in reader:
        print(r['c.uid'],r['c.name'], r['c.description'])
        hex_rgb_color=r['c.description']
        # rgb_color =
        # color_str = f'hsl({c}, 100%, 50%)'
        color_str = hex_rgb_color
        # colorsys.rgb_to_hls()
        html+=f'''
        <tr>
            <td>{r['c.uid']}</td>
            <td style="background-color:{color_str}; height:50px; vertical-align:top">
            <span style="background-color:white; display:block">{r['c.name']}</span>
            </td>
            <td>{color_str}</td>
        </tr>'''

html+='</tbody></table>'


display(HTML(html))

['c.uid', 'c.name', 'c.description']
lightYellowCharacteristic light yellow #fcf0be
mediumBlueCharacteristic medium blue #4cb3d4
lightBlueCharacteristic light blue #a0cedb
mediumGreenCharacteristic medium green #3cbc65
lightGreenCharacteristic light green #a2cfaf
darkRedCharacteristic dark red #bc2c14
ochreCharacteristic ochre #cab91d
blackCharacteristic black #474747
greyCharacteristic grey grey
brownCharacteristic brown brown
darkBlueCharacteristic dark blue #441cb3
darkGreenCharacteristic dark green #046424
pinkCharacteristic pink pink
redCharacteristic red red
orangeCharacteristic orange orange
yellowCharacteristic yellow yellow
whiteCharacteristic white #fcf0be


uid,name,color
lightYellowCharacteristic,light yellow,#fcf0be
mediumBlueCharacteristic,medium blue,#4cb3d4
lightBlueCharacteristic,light blue,#a0cedb
mediumGreenCharacteristic,medium green,#3cbc65
lightGreenCharacteristic,light green,#a2cfaf
darkRedCharacteristic,dark red,#bc2c14
ochreCharacteristic,ochre,#cab91d
blackCharacteristic,black,#474747
greyCharacteristic,grey,grey
brownCharacteristic,brown,brown
